# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
weather_df = pd.read_csv('../weather_data.csv')
weather_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yellowknife,62.4560,-114.3525,17.60,49,75,13.80,CA,1619315431
1,ushuaia,-54.8000,-68.3000,50.00,50,20,2.30,AR,1619315713
2,mataura,-46.1927,168.8643,45.00,82,38,8.99,NZ,1619315176
3,albany,42.6001,-73.9662,55.00,38,100,1.92,US,1619315499
4,seoul,37.5683,126.9778,66.20,45,0,13.80,KR,1619315439
5,panzhihua,26.5851,101.7128,80.71,22,23,0.13,CN,1619315888
6,gisborne,-38.6533,178.0042,68.00,45,5,1.99,NZ,1619315888
7,kununurra,-15.7667,128.7333,84.20,24,100,14.97,AU,1619315889
8,qaanaaq,77.4840,-69.3632,23.56,68,91,8.08,GL,1619315889
9,plaisance-du-touch,43.5657,1.2975,55.00,66,0,9.22,FR,1619315643


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
figure_layout = {'width': '600px', 'height': '440px'}
fig = gmaps.figure(layout=figure_layout)
heatmap_layer=gmaps.heatmap_layer(weather_data[['Lat', 'Lng']], weights=weather_data['Humidity'], 
                                  max_intensity=100, point_radius=9)
heatmap_layer.opacity = .75
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='440px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Enter a few weather criteria to get ideal weather locations
temp_ideal = weather_data.loc[weather_data['Max Temp'] > 75]
humidity_ideal = temp_ideal.loc[temp_ideal['Humidity'] < 75]
wind_ideal = humidity_ideal.loc[humidity_ideal['Wind Speed'] < 5]
hotel_df = wind_ideal.loc[wind_ideal['Cloudiness'] < 20]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
189,garhi khairo,28.0606,67.9803,84.61,20,0,4.32,PK,1619315956
320,bambanglipuro,-7.9500,110.2833,82.76,66,7,3.38,ID,1619315733
370,bannu,32.9854,70.6027,77.22,31,3,1.07,PK,1619316033
408,badarpur,24.9000,92.6000,88.50,25,7,1.88,IN,1619316048
410,kulhudhuffushi,6.6221,73.0700,83.59,70,17,2.75,MV,1619315757
423,tura,25.5198,90.2201,86.94,26,3,3.27,IN,1619315762
472,manama,26.2154,50.5832,82.40,54,0,4.61,BH,1619316078
484,turbat,26.0023,63.0440,81.36,54,3,1.68,PK,1619316074
488,khandbari,27.3747,87.2039,77.83,22,2,1.19,NP,1619316085
519,tezu,27.9167,96.1667,89.56,14,6,1.59,IN,1619315563


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df.insert(9, "Hotel Name", "")
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
189,garhi khairo,28.0606,67.9803,84.61,20,0,4.32,PK,1619315956,
320,bambanglipuro,-7.9500,110.2833,82.76,66,7,3.38,ID,1619315733,
370,bannu,32.9854,70.6027,77.22,31,3,1.07,PK,1619316033,
408,badarpur,24.9000,92.6000,88.50,25,7,1.88,IN,1619316048,
410,kulhudhuffushi,6.6221,73.0700,83.59,70,17,2.75,MV,1619315757,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
figure_layout = {'width': '600px', 'height': '440px'}
fig = gmaps.figure(layout=figure_layout)

heatmap_layer=gmaps.heatmap_layer(weather_data[['Lat', 'Lng']], weights=weather_data['Humidity'], 
                                  max_intensity=100, point_radius=9)
locations_layer=gmaps.locationslayer(locations, info_box_content=hotel_info)
heatmap_layer.opacity = .5
fig.add_layer(heatmap_layer)
fig.add_layer(locations_layer)
fig

# Display figure
